In [8]:
import os
import json
import faiss
import numpy as np
from openai import OpenAI
import traceback
import openai  # 추가
from dotenv import load_dotenv
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch

from datasets import load_dataset

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

client_gpt = OpenAI()

In [3]:
with open("/upstage-ai-advanced-ir7/data/eval.jsonl", "r") as f:
    eval_doc_mapping = [json.loads(line) for line in f]
    

doc_mapping = {}
with open("/upstage-ai-advanced-ir7/data/documents.jsonl", "r") as f:
    for line in f:
        doc = json.loads(line)
        doc_mapping[doc['docid']] = doc 

index = faiss.read_index("knn_index_cosine.faiss")

# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()
    
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_path = 'Dongjin-kr/ko-reranker'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to('cuda')

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

NameError: name 'model_name' is not defined

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch.nn as nn

# Load the pre-trained model and tokenizer
model_path = 'Dongjin-kr/ko-reranker'
model = AutoModelForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True)

# Reinitialize the classification layer to match num_labels=2
model.classifier.out_proj = nn.Linear(model.config.hidden_size, 2)  # For binary classification

tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example data: queries and documents with binary relevance labels
data = {
    "query": ["What is meditation?", "What are pandas?"],
    "document": ["Meditation helps reduce stress.", "Pandas are black and white bears."],
    "label": [1, 0]  # 1: relevant, 0: irrelevant
}

# Convert the data into a Hugging Face dataset
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['query'], examples['document'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the data into training and evaluation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Trainer with CrossEntropyLoss by default for classification
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start fine-tuning
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch.nn as nn
import torch

# Load the pre-trained model and tokenizer
model_path = 'Dongjin-kr/ko-reranker'
model = AutoModelForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True)

# Reinitialize the classification layer to match num_labels=2
model.classifier.out_proj = nn.Linear(model.config.hidden_size, 2)  # For binary classification

tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example data: queries and documents with binary relevance labels
data = {
    "query": ["What is meditation?", "What are pandas?"],
    "document": ["Meditation helps reduce stress.", "Pandas are black and white bears."],
    "label": [1, 0]  # 1: relevant, 0: irrelevant
}

# Convert the data into a Hugging Face dataset
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['query'], examples['document'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Ensure that labels are in Long format for CrossEntropyLoss
def convert_labels_to_long(examples):
    examples['label'] = [int(x) for x in examples['label']]  # Ensure labels are integers (Long)
    return examples

tokenized_dataset = tokenized_dataset.map(convert_labels_to_long, batched=True)

# Split the data into training and evaluation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Trainer with CrossEntropyLoss by default for classification
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start fine-tuning
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch.nn as nn
import torch

# Load the pre-trained model and tokenizer
model_path = 'Dongjin-kr/ko-reranker'
model = AutoModelForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True)

# Reinitialize the classification layer to match num_labels=2
model.classifier.out_proj = nn.Linear(model.config.hidden_size, 2)  # For binary classification

tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example data: queries and documents with binary relevance labels
data = {
    "query": ["What is meditation?", "What are pandas?"],
    "document": ["Meditation helps reduce stress.", "Pandas are black and white bears."],
    "label": [1, 0]  # 1: relevant, 0: irrelevant
}

# Convert the data into a Hugging Face dataset
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['query'], examples['document'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Ensure that labels are in Long format for CrossEntropyLoss
def convert_labels_to_long(examples):
    examples['label'] = torch.tensor(examples['label'], dtype=torch.long)  # Ensure labels are integers (Long)
    return examples

tokenized_dataset = tokenized_dataset.map(convert_labels_to_long, batched=True)

# Split the data into training and evaluation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Trainer with CrossEntropyLoss by default for classification
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start fine-tuning
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch.nn as nn
import torch

# Load pre-trained model and tokenizer
model_path = 'Dongjin-kr/ko-reranker'
model = AutoModelForSequenceClassification.from_pretrained(model_path, ignore_mismatched_sizes=True)

# Reinitialize the classification layer to match num_labels=2
model.classifier.out_proj = nn.Linear(model.config.hidden_size, 2)  # For binary classification

tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example data: queries and documents with binary relevance labels
data = {
    "query": ["What is meditation?", "What are pandas?"],
    "document": ["Meditation helps reduce stress.", "Pandas are black and white bears."],
    "label": [1, 0]  # 1: relevant, 0: irrelevant
}

# Convert the data into a Hugging Face dataset
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['query'], examples['document'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Ensure that labels are in Long format for CrossEntropyLoss
def convert_labels_to_long(examples):
    examples['label'] = torch.tensor(examples['label'], dtype=torch.long)  # Ensure labels are integers (Long)
    return examples

tokenized_dataset = tokenized_dataset.map(convert_labels_to_long, batched=True)

# Split the data into training and evaluation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Trainer with CrossEntropyLoss by default for classification
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Ensure proper gradient computation by explicitly casting tensors if necessary
def custom_training_step(model, inputs):
    model.train()
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    # CrossEntropyLoss expects labels in Long format and logits in Float format
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(logits, inputs["labels"].long())  # Ensure labels are long integers
    return loss

# Manually set the training step to avoid dtype mismatch
trainer.training_step = custom_training_step

# Start fine-tuning
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch,Training Loss,Validation Loss
1,No log,0.212737
2,No log,0.212737
3,No log,0.212737


/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/envs/ir/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

TrainOutput(global_step=3, training_loss=0.7478123505910238, metrics={'train_runtime': 14.0905, 'train_samples_per_second': 0.213, 'train_steps_per_second': 0.213, 'total_flos': 698948522496.0, 'train_loss': 0.7478123505910238, 'epoch': 3.0})